# Generating routes

This notebook demonstrates how to generate routes from isochrones to a point.


## Step 1: Import the googleroutes library and access api key

In [ ]:
%pip install googleroutes
%pip install python-dotenv # OPTIONAL - for loading environment variables

In [1]:
import os
from dotenv import load_dotenv # OPTIONAL - for loading environment variables

In [2]:
# This is optional, you can use your preferred method to load the api key
load_dotenv()
api_key = os.getenv("API_KEY")

 ## Step 2: initialise the client using your api key
When starting the client, the global CRS is set. You may wish to specify this, however the default is EPSG:4326 which is what the GoogleMaps API uses.

In [3]:
import googleroutes as gr

gr.start_client(api_key, crs="EPSG:4326")

Google Maps client started.


## Step 3: Load in the places dataframe and isochrones dataframe

In [7]:
import geopandas as gpd

places = gpd.read_file('places.gpkg')
isochrones = gpd.read_file('isochrones.gpkg')

## Step 4: Merge the schools and isochrones dataframes on the school names
This allows us to call the generator function on this merged dataframe. \par
The geometry column of merged_df should be the isochrones geometry, the school coords can be accessed from the isochrones dataframe.

In [8]:
merged_df = isochrones.merge(places[['place', ]]) # merge on school name

In [10]:
merged_df # geometry column should be the isochrones geometry

,place,CENTER_LON,CENTER_LAT,AA_METERS,AA_MODE,TOTAL_POP,geometry
0,Hicks Building,-1.4864656622895907,53.38123290077746,1000,foot-walking,27508,"POLYGON ((-1.50081 53.37849, -1.50029 53.37709..."
1,Town Hall,-1.4721797331394337,53.38037667966202,1000,foot-walking,24151,"POLYGON ((-1.48716 53.37802, -1.48715 53.37795..."
2,Cathedral,-1.4693382,53.3830177,1000,foot-walking,20865,"POLYGON ((-1.48472 53.38114, -1.4847 53.38105,..."


## Step 5: generating routes and how to format the input
The following are required for the generator function:
- A geometry column in the dataframe that contains the isochrones geometry
- A column in the dataframe that contains the place name
- 2 columns containing the longitude and latitude of the place coords

Considerations:
- Supply the coords column names as a tuple of (longitude, latitude) : if you have generated the isochrones using the OpenRouteService isochrone generator, the column names will be 'CENTER_LON' and 'CENTER_LAT'
- If the dataframe contains a geopandas geometry column, this will be used as the geometry column
- The routes_output must be a directory as multiple files will be generated



In [12]:
gr.generate_from_isochrones(dataframe=merged_df, schools_column='place', school_coords_column=('CENTER_LON', 'CENTER_LAT'), mode='walking', driver='GPKG', routes_output='example_routes')

The routes are automatically saved in the specified directory, in seperate files for each place. \
To view the routes in QGIS, you can use the following script in the QGIS python scripter to load the files into a new group:

In [13]:
import os

root = 'path/to/folder/containing/routes'
proj = QgsProject.instance().layerTreeRoot()

group = proj.insertGroup(0, 'YOUR_GROUP_NAME')

for file in os.listdir(root):
    if file == 'desktop.ini': # specific to our project which was saved on GoogleDrive which creates this file
        continue
    if not file.endswith('.gpkg'): # change to .shp if using shapefiles or .geojson if using geojson files
        continue
    print(file)
    layer = QgsVectorLayer(f'{root}/{file}', file[:-4], 'ogr') # file[:-4] removes the .gpkg extension. Change to file[:-3] if using .shp files or file[:-7] for .geojson files
    QgsProject.instance().addMapLayer(layer, False)
    group.insertChildNode(1, QgsLayerTreeLayer(layer))


# NOTE: The above code is specific to the QGIS python scripter and may not work in other environments.



NameError: name 'QgsProject' is not defined